## Import the libaries that will be used and make sure to enable large datasets for altair 

In [65]:
import pandas as pd 
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Import the data and look to see if loaded

In [39]:
data = pd.read_csv("denver-1.nov.csv")
data.head()

,raw_row_number,date,time,location,lat,lng,district,precinct,type,disposition,arrest_made,citation_issued,warning_issued,outcome
0,1,2012-11-13,00:02:22,8500 W CRESTLINE AVE,39.618329,-105.092691,4,423,vehicular,K - Street Check Completed,False,False,False,NaN
1,16,2012-11-29,00:25:01,8500 W CRESTLINE AVE,39.618329,-105.092691,4,423,vehicular,Party Advised,False,False,False,NaN
2,72,2012-11-23,22:58:10,4830 S HOLLAND WAY,39.630666,-105.102551,4,423,vehicular,K - Street Check Completed,False,False,False,NaN
3,121,2012-11-27,17:48:33,4885 S QUEBEC ST,39.626174,-104.904062,3,324,pedestrian,In Service,False,False,False,NaN
4,138,2012-11-14,23:46:58,W LAYTON AVE / S WADSWORTH BLVD,39.630150,-105.081693,4,423,vehicular,Party Advised,False,False,False,NaN


# Remove NaN variables 

In [119]:
data = data[data["precinct"] != 'None']
data.head()

# These statistics get you the average issued or arrest  by the Denver Police precincts

### Average warnings issued 11.4%

In [121]:
data["warning_issued"].sum()/data["raw_row_number"].count()

0.1143732860138732

### Average arrests made was 15.5%

In [122]:
data["arrest_made"].sum()/data["raw_row_number"].count()

0.15518632037425392

### Average citations issued were 16.9% 

In [123]:
data["citation_issued"].sum()/data["raw_row_number"].count()

0.16922084207130184

# Create pivot tables of arrests, citations, and warnings to understand precinct habits and visualize data 

In [260]:
piv_arrests = pd.pivot_table(data,
                                     values=['arrest_made'],
                                     index='precinct',
                                     aggfunc='sum'
                                    )


arrests =pd.DataFrame(piv_arrests["arrest_made"])
arrests = arrests.reset_index()
arrests.head()

,precinct,arrest_made
0,111,41.0
1,112,40.0
2,113,14.0
3,121,39.0
4,122,40.0


In [261]:
piv_warning = pd.pivot_table(data,
                                     values=['warning_issued'],
                                     index='precinct',
                                     aggfunc='sum'
                                    )


warnings =pd.DataFrame(piv_warning["warning_issued"])
warnings = warnings.reset_index()
warnings.head()

,precinct,warning_issued
0,111,33.0
1,112,8.0
2,113,27.0
3,121,25.0
4,122,26.0


In [262]:
piv_citation = pd.pivot_table(data,
                                     values=['citation_issued'],
                                     index='precinct',
                                     aggfunc='sum'
                                    )


citations =pd.DataFrame(piv_citation["citation_issued"])
citations = citations.reset_index()
citations.head()

,precinct,citation_issued
0,111,53.0
1,112,36.0
2,113,30.0
3,121,36.0
4,122,26.0


# Merge the data to one Dataframe to use in to visualize in altair

In [267]:
merged_data = pd.merge(arrests,citations, on=['precinct'])
final_data = pd.merge(merged_data,warnings, on=["precinct"])
final_data.head()

,precinct,arrest_made,citation_issued,warning_issued
0,111,41.0,53.0,33.0
1,112,40.0,36.0,8.0
2,113,14.0,30.0,27.0
3,121,39.0,36.0,25.0
4,122,40.0,26.0,26.0


# Sort the dataframe to show highest arrests, citations, and warnings issued.

In [319]:
final_data.sort_values(by=['arrest_made'],ascending=False).head()

,precinct,arrest_made,citation_issued,warning_issued
11,223,78.0,60.0,46.0
6,211,65.0,24.0,19.0
20,411,54.0,46.0,33.0
32,621,50.0,26.0,16.0
12,311,49.0,49.0,42.0


In [320]:
final_data.sort_values(by=['citation_issued'],ascending=False).head()

,precinct,arrest_made,citation_issued,warning_issued
11,223,78.0,60.0,46.0
13,312,19.0,59.0,24.0
0,111,41.0,53.0,33.0
12,311,49.0,49.0,42.0
20,411,54.0,46.0,33.0


In [321]:
final_data.sort_values(by=['warning_issued'],ascending=False).head()

,precinct,arrest_made,citation_issued,warning_issued
11,223,78.0,60.0,46.0
12,311,49.0,49.0,42.0
22,421,26.0,40.0,37.0
0,111,41.0,53.0,33.0
20,411,54.0,46.0,33.0


# Altair visuals for the web 

In [323]:
arrests = alt.Chart(final_data).mark_circle(color="red").encode(
    x='precinct:N',
    y='sum(arrest_made):Q',
    tooltip=["arrest_made:Q","precinct:N"]
   
).transform_fold(
    fold=['arrest_made'])
citation = alt.Chart(final_data).mark_circle(color="yellow").encode(
    x='precinct:N',
    y='sum(citation_issued):Q',
    tooltip=["citation_issued:Q","precinct:N"]
   
).transform_fold(
    fold=['citation_issued'])
warning = alt.Chart(final_data).mark_circle(color="green").encode(
    x='precinct:N',
    y='sum(warning_issued):Q',
    tooltip=["warning_issued:Q","precinct:N"]
   
).transform_fold(
    fold=['warning_issued'])
final = arrests +citation +warning
final.save("final_vis.html",embed_options={"renderer":"svg"})

In [329]:
background = alt.Chart(data).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)
selector = alt.selection_single(empty='all', fields=['arrest_made'])

color_condition = alt.condition(selector,"arrest_made",       
                                  alt.value("lightgray"))
points = alt.Chart(data).mark_circle().encode(
    longitude='lng:Q',
    latitude='lat:Q',
    size=alt.value(10),
    color =color_condition,
    tooltip=["precinct","arrest_made:Q"]
).add_selection(selector)
arrest= background+ points
arrest.save("arrest.html",embed_options={"renderer":"svg"})

In [324]:
background = alt.Chart(data).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)
selector = alt.selection_single(empty='all', fields=['arrest_made'])

color_condition = alt.condition(selector,"precinct",       
                                  alt.value("lightgray"))
points = alt.Chart(data).mark_circle().encode(
    longitude='lng:Q',
    latitude='lat:Q',
    size=alt.value(20),
    color =color_condition,
    tooltip=["precinct","arrest_made:Q"]
).add_selection(selector)
arrests_precinct = background + points
arrests_precinct.save("arrests_precinct.html",embed_options={"renderer":"svg"})

In [325]:
background = alt.Chart(data).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)
selector = alt.selection_single(empty='all', fields=['citation_issued'])

color_condition = alt.condition(selector,"citation_issued",       
                                  alt.value("lightgray"))
points = alt.Chart(data).mark_circle().encode(
    longitude='lng:Q',
    latitude='lat:Q',
    size=alt.value(10),
    color =color_condition,
    tooltip=["precinct","citation_issued:Q"]
).add_selection(selector)
citations = background+ points
citations.save("citations.html",embed_options={"renderer":"svg"})

In [326]:
background = alt.Chart(data).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)
selector = alt.selection_single(empty='all', fields=['citation_issued'])

color_condition = alt.condition(selector,"precinct",       
                                  alt.value("lightgray"))
points = alt.Chart(data).mark_circle().encode(
    longitude='lng:Q',
    latitude='lat:Q',
    size=alt.value(20),
    color =color_condition,
    tooltip=["precinct","citation_issued:Q"]
).add_selection(selector)
citations_precinct = background + points
citations_precinct.save("citations_precinct.html",embed_options={"renderer":"svg"})

In [327]:
background = alt.Chart(data).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)
selector = alt.selection_single(empty='all', fields=['warning_issued'])

color_condition = alt.condition(selector,"warning_issued",       
                                  alt.value("lightgray"))
points = alt.Chart(data).mark_circle().encode(
    longitude='lng:Q',
    latitude='lat:Q',
    size=alt.value(10),
    color =color_condition,
    tooltip=["precinct","warning_issued:Q"]
).add_selection(selector)
warnings = background+ points
warnings.save("warnings.html",embed_options={"renderer":"svg"})

In [328]:
background = alt.Chart(data).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)
selector = alt.selection_single(empty='all', fields=['warning_issued'])

color_condition = alt.condition(selector,"precinct",       
                                  alt.value("lightgray"))
points = alt.Chart(data).mark_circle().encode(
    longitude='lng:Q',
    latitude='lat:Q',
    size=alt.value(20),
    color =color_condition,
    tooltip=["precinct","warning_issued:Q"]
).add_selection(selector)
warnings_precinct = background + points
warnings_precinct.save("warnings_precinct.html",embed_options={"renderer":"svg"})